In [1]:
import cv2
import mediapipe as mp

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

hands = mp_hands.Hands(
    model_complexity=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7
)

# Define gesture recognition logic
def recognize_gesture(landmarks):
    thumb_tip = landmarks[mp_hands.HandLandmark.THUMB_TIP]
    index_tip = landmarks[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    middle_tip = landmarks[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
    ring_tip = landmarks[mp_hands.HandLandmark.RING_FINGER_TIP]
    pinky_tip = landmarks[mp_hands.HandLandmark.PINKY_TIP]
    
    thumb_ip = landmarks[mp_hands.HandLandmark.THUMB_IP]
    index_pip = landmarks[mp_hands.HandLandmark.INDEX_FINGER_PIP]
    middle_pip = landmarks[mp_hands.HandLandmark.MIDDLE_FINGER_PIP]
    ring_pip = landmarks[mp_hands.HandLandmark.RING_FINGER_PIP]
    pinky_pip = landmarks[mp_hands.HandLandmark.PINKY_PIP]

    # Victory Gesture (V-sign)
    if index_tip.y < index_pip.y and middle_tip.y < middle_pip.y and ring_tip.y > ring_pip.y and pinky_tip.y > pinky_pip.y:
        return "Victory"

    # Call Gesture (Thumb and Pinky Extended)
    if thumb_tip.y < thumb_ip.y and pinky_tip.y < pinky_pip.y and index_tip.y > index_pip.y and middle_tip.y > middle_pip.y and ring_tip.y > ring_pip.y:
        return "Call"

    # Stop Gesture (Hand open, fingers straight)
    if thumb_tip.y < thumb_ip.y and index_tip.y < index_pip.y and middle_tip.y < middle_pip.y and ring_tip.y < ring_pip.y and pinky_tip.y < pinky_pip.y:
        return "Stop"

    # Thumbs Down Gesture (Thumb down, fingers closed)
    if thumb_tip.y > thumb_ip.y and thumb_tip.x < thumb_ip.x and index_tip.y > index_pip.y:
        return "Thumbs Down"

    # Thumbs Up Gesture (Thumb up, fingers closed)
    if thumb_tip.y < thumb_ip.y and thumb_tip.x < thumb_ip.x and index_tip.y > index_pip.y:
        return "Thumbs Up"

    # Sorry Gesture (Right hand touching left chest)
    if all(landmark.y < thumb_ip.y for landmark in [thumb_tip, index_tip, middle_tip, ring_tip, pinky_tip]) and thumb_tip.x < index_tip.x:
        return "Sorry"

    # Thank You Gesture (Flat hand, fingers together, moving away from the face)
    if all(landmark.y < thumb_ip.y for landmark in [thumb_tip, index_tip, middle_tip, ring_tip, pinky_tip]) and abs(thumb_tip.x - index_tip.x) < 0.02:
        return "Thank You"

    return "Unknown Gesture"

# Start webcam capture
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    # Flip the image horizontally for a later selfie-view display, and convert the BGR image to RGB.
    frame = cv2.flip(frame, 1)
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the image and detect hand landmarks.
    result = hands.process(image)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            # Recognize gesture
            gesture = recognize_gesture(hand_landmarks.landmark)
            cv2.putText(frame, gesture, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Display the output
    cv2.imshow('Hand Gesture Recognition', frame)

    # Exit when 'q' key is pressed
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


C:\Users\DELL\New folder\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
